# Exploratory Data Analysis (EDA): Training Data

## Overview
This notebook performs an initial exploratory data analysis on the training dataset (`train.xlsx`) for the property valuation project. 

**Goals:**
1. Inspect data structure and types.
2. Identify missing values and potential data quality issues.
3. Analyze distributions of key variables (Price, Sqft, etc.).
4. Explore correlations between features and the target variable (Price).
5. Visualize geospatial patterns.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set visualization style
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

## 1. Load Data
Loading the Excel file into a pandas DataFrame.

In [ ]:
# Load the dataset
file_path = '../data/raw/train.xlsx'  # Adjust path if necessary
try:
    df = pd.read_excel(file_path)
    print("Data loaded successfully.")
    print(f"Shape: {df.shape}")
except FileNotFoundError:
    print(f"File not found at {file_path}. Please check the path.")
except Exception as e:
    print(f"An error occurred: {e}")

## 2. Data Inspection
Checking the first few rows, data types, and summary statistics.

In [ ]:
# Display first 5 rows
display(df.head())

In [ ]:
# Check data types and non-null counts
df.info()

In [ ]:
# Summary statistics for numerical columns
display(df.describe().T)

## 3. Missing Value Analysis
Identifying columns with missing data to determine cleaning strategy.

In [ ]:
# Calculate missing values
missing = df.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)

if not missing.empty:
    plt.figure(figsize=(10, 6))
    sns.barplot(x=missing.values, y=missing.index, palette='viridis')
    plt.title('Missing Values per Column')
    plt.xlabel('Count of Missing Values')
    plt.show()
    display(missing)
else:
    print("No missing values found in the dataset.")

## 4. Target Variable Analysis: Price
Analyzing the distribution of the target variable `price`.

In [ ]:
# Plot Price Distribution
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
sns.histplot(df['price'], bins=50, kde=True, color='blue')
plt.title('Distribution of Price')
plt.xlabel('Price')

plt.subplot(1, 2, 2)
sns.boxplot(x=df['price'], color='blue')
plt.title('Boxplot of Price')
plt.xlabel('Price')

plt.tight_layout()
plt.show()

**Log-Transformation:**
Since property prices often follow a right-skewed distribution, applying a log transformation can help normalize the data for better model performance.

In [ ]:
# Log-transform Price
df['log_price'] = np.log1p(df['price'])

plt.figure(figsize=(14, 6))
sns.histplot(df['log_price'], bins=50, kde=True, color='green')
plt.title('Distribution of Log-Price')
plt.xlabel('Log(Price)')
plt.show()

## 5. Feature Analysis
Exploring relationships between key features (e.g., `sqft_living`, `grade`) and `price`.

In [ ]:
# Scatter plot: Sqft Living vs. Price
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='sqft_living', y='log_price', alpha=0.5)
plt.title('Sqft Living vs. Log-Price')
plt.show()

In [ ]:
# Boxplot: Grade vs. Log-Price
plt.figure(figsize=(12, 6))
sns.boxplot(data=df, x='grade', y='log_price', palette='coolwarm')
plt.title('Grade vs. Log-Price')
plt.show()

## 6. Correlation Analysis
Checking correlation between numerical features and the target variable.

In [ ]:
# Compute correlation matrix
numeric_cols = df.select_dtypes(include=[np.number]).columns
corr_matrix = df[numeric_cols].corr()

# Heatmap
plt.figure(figsize=(14, 12))
sns.heatmap(corr_matrix, annot=False, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
# Top correlated features with Price
price_corr = corr_matrix['price'].sort_values(ascending=False)
print("Top 10 features correlated with Price:")
display(price_corr.head(10))

## 7. Geospatial Analysis
Visualizing property locations using Latitude and Longitude.

In [ ]:
if 'lat' in df.columns and 'long' in df.columns:
    plt.figure(figsize=(10, 10))
    sns.scatterplot(x='long', y='lat', hue='log_price', data=df, 
                    palette='viridis', alpha=0.6, s=10)
    plt.title('Geospatial Distribution of Properties (Color by Log-Price)')
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.show()
else:
    print("Latitude/Longitude columns not found.")

## 8. Conclusion
Summary of findings from the EDA:
- **Data Quality:** (Note any missing values or anomalies)
- **Target Distribution:** (Note skewness of price)
- **Key Drivers:** (Note features with high correlation like sqft_living, grade)
- **Next Steps:** Proceed to feature engineering (handling dates, encoding categorical variables) and merging with satellite data.